In [19]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output

sub_path = "../submissions"
all_files = os.listdir(sub_path)

print(all_files)

['sub_200_ens_densenet.csv', 'submission38.csv', 'sub_TF_keras.csv', 'submission43.csv']


In [20]:
# Read and concatenate submissions
out1 = pd.read_csv("../submissions/sub_200_ens_densenet.csv", index_col=0)
out2 = pd.read_csv("../submissions/sub_TF_keras.csv", index_col=0)
out3 = pd.read_csv("../submissions/submission38.csv", index_col=0)
out4 = pd.read_csv("../submissions/submission43.csv", index_col=0)
concat_sub = pd.concat([out1, out2, out3, out4], axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
0,5941774d,1.943216e-02,0.005586,0.008875,0.005586
1,4023181e,3.168809e-02,0.145927,0.146779,0.031688
2,b20200e4,4.000000e-08,0.000015,0.048646,0.001000
3,e7f018bb,9.925741e-01,0.999914,0.999184,0.999000
4,4371c8c3,2.215107e-02,0.033843,0.277251,0.022151


In [21]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [22]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [23]:
sub_base = pd.read_csv('../submissions/submission43.csv')

In [24]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:4] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:4] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv('submission54.csv', 
                                        index=False, float_format='%.6f')
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg_base,is_iceberg
0,5941774d,1.943216e-02,0.005586,0.008875,0.005586,0.019432,5.586000e-03,0.011782,0.008875,0.005586,0.005586
1,4023181e,3.168809e-02,0.145927,0.146779,0.031688,0.146779,3.168800e-02,0.100572,0.145927,0.031688,0.031688
2,b20200e4,4.000000e-08,0.000015,0.048646,0.001000,0.048646,4.000000e-08,0.019662,0.001000,0.001000,0.001000
3,e7f018bb,9.925741e-01,0.999914,0.999184,0.999000,0.999914,9.925741e-01,0.998117,0.999184,0.999000,0.999000
4,4371c8c3,2.215107e-02,0.033843,0.277251,0.022151,0.277251,2.215100e-02,0.126529,0.033843,0.022151,0.022151
